In [1]:
#hide
!pip install -Uqq fastbook
import fastbook
fastbook.setup_book()

C:\Users\arek\anaconda3\envs\ml\lib\site-packages\fastbook\__init__.py:22: UserWarning: Missing `graphviz` - please run `conda install fastbook`
  except ModuleNotFoundError: warn("Missing `graphviz` - please run `conda install fastbook`")


In [2]:
import pandas as pd
from pathlib import Path
from fastai.vision.all import *
from fastai.data.all import*
from fastbook import *
path = "./skyhacks_hackathon_dataset"
labels = pd.read_csv(path+"/training_labels.csv")
categories = list(labels.columns[1:])

In [3]:
def extract_labels(row):
    fastai_labels = ''
    for category in categories:
        if(row[category] == True):
            fastai_labels += (category + " ")
    return str(fastai_labels).strip()

In [7]:
labels['fastai'] = labels.apply(extract_labels,axis=1)


In [8]:
path_train = path+"/training_images/"
def get_x(r):
    return path_train+str(r['Name'])
def get_y(r): return r['fastai'].split(' ')

In [9]:
dblock = DataBlock(blocks=(ImageBlock, MultiCategoryBlock),
                   get_x = get_x,
                   get_y = get_y,
                   splitter=RandomSplitter(), 
                   item_tfms = RandomResizedCrop(128, min_scale=0.35),
                   batch_tfms=aug_transforms(mult=2))
                    #flip_vert=True
#dsets = dblock.datasets(labels)
dls = dblock.dataloaders(labels,num_workers=0)

In [10]:
def accuracy_multi(inp, targ, thresh=0.5, sigmoid=True):
    "Compute accuracy when `inp` and `targ` are the same size."
    if sigmoid: inp = inp.sigmoid()
    return ((inp>thresh)==targ.bool()).float().mean()

In [11]:
import fastai.metrics as met
print(len(categories))

38


In [9]:
res50 = cnn_learner(dls, resnet50,metrics=partial(accuracy_multi, thresh=0.2))
res50.fine_tune(6, base_lr=3e-3, freeze_epochs=2)

epoch,train_loss,valid_loss,accuracy_multi,time
0,0.943441,0.706062,0.215210,01:57
1,0.630929,0.217855,0.868297,01:53


epoch,train_loss,valid_loss,accuracy_multi,time
0,0.247464,0.186547,0.907665,01:59
1,0.222796,0.178450,0.907857,01:58
2,0.203489,0.172456,0.913148,01:57
3,0.188627,0.165029,0.912874,01:56
4,0.177097,0.163465,0.915150,01:57
5,0.171300,0.163332,0.913121,01:56


In [10]:
res34 = cnn_learner(dls, resnet34, metrics=partial(accuracy_multi, thresh=0.2))
res34.fine_tune(6, base_lr=3e-3, freeze_epochs=2)

epoch,train_loss,valid_loss,accuracy_multi,time
0,0.969503,0.708075,0.190070,01:50
1,0.647606,0.231124,0.850422,01:50


epoch,train_loss,valid_loss,accuracy_multi,time
0,0.255588,0.193674,0.901853,01:53
1,0.232695,0.184456,0.900428,01:55
2,0.213547,0.179238,0.905171,01:54
3,0.199839,0.174438,0.908817,01:53
4,0.189499,0.170596,0.908214,01:51
5,0.182538,0.169453,0.907720,01:50


In [11]:
res101 = cnn_learner(dls, resnet101, metrics=partial(accuracy_multi, thresh=0.2))
res101.fine_tune(6, base_lr=3e-3, freeze_epochs=2)

epoch,train_loss,valid_loss,accuracy_multi,time
0,0.936036,0.698360,0.223051,02:02
1,0.628000,0.220616,0.864020,02:02


epoch,train_loss,valid_loss,accuracy_multi,time
0,0.244341,0.188604,0.901716,02:09
1,0.222760,0.185829,0.909667,02:08
2,0.204370,0.182100,0.909612,02:10
3,0.190185,0.169497,0.913532,02:09
4,0.178302,0.166823,0.915671,02:12
5,0.170136,0.164746,0.914026,02:10


In [12]:
def ensemble_predition(img):
    resnet50_predicition = [int(x) for x in list(res50.predict(img))[1]]
    resnet34_predicition = [int(x) for x in list(res34.predict(img))[1]]
    resnet101_predicition = [int(x) for x in list(res101.predict(img))[1]]
    sum_pred = [x[0]+x[1]+x[2] for x in zip(resnet50_predicition,resnet34_predicition,resnet101_predicition)]
    prediction = [int(round(x/3)) for x in sum_pred]
    return prediction

In [13]:
def compare(img):
    preds = ensemble_predition(img[0])
    truth = [int(x) for x in list(img[1])]
    return  f1_score(truth, preds, average='weighted')

    

In [14]:
from sklearn.metrics import f1_score
import tqdm
dsets = dblock.datasets(labels)

In [15]:
# scores_ensemble = []
# for dset in tqdm.tqdm(dsets):
#     scores_ensemble.append(compare(dset))

In [16]:
from PIL import Image
path = "C:\\Users\\arek\\Desktop\\hackathon-ai\\skyhacks_hackathon_dataset\\live_test_images"

In [17]:
list_of_images = []
for imageName in os.listdir(path):
    list_of_images.append(imageName)

In [18]:
result = res34.predict(path + "\\" + list_of_images[0])

In [19]:
def ensemble_predition(path,treshold=0):
    resnet50_predicition = res50.predict(path)[0]
    resnet34_predicition = res34.predict(path)[0]
    resnet101_predicition = res101.predict(path)[0]
    sum_pred = resnet50_predicition+resnet34_predicition+resnet101_predicition
    set_ = set(sum_pred)
    json_of_preds  = {}
    for key in set_:
        if sum_pred.count(key) > treshold:
            json_of_preds[key] = sum_pred.count(key) 
    print(json_of_preds)
    return json_of_preds

In [20]:
result_dict = {}
for img in list_of_images:
    result_dict[path + "\\"+img] ={"result": ensemble_predition(path + "\\" + img), "image": Image.open(path + "\\"+img) }

{'Building': 2, 'Furniture': 1, 'Museum': 1}


{'Building': 3, 'Motor': 1, 'Person': 3}


{'Forest': 2, 'Landscape': 2, 'Plants': 3, 'Grass': 3, 'Trees': 3}


{'Building': 3, 'Windows': 3, 'City': 1, 'Road': 3, 'Footpath': 3}


{'Building': 3, 'Trees': 3, 'Plants': 1, 'Grass': 3}


...

{'Plants': 2, 'Trees': 3, 'Forest': 3, 'Grass': 3}


{'Building': 3, 'Plants': 2, 'Grass': 3, 'Windows': 3, 'Trees': 3, 'Footpath': 2}


{'Forest': 1, 'Landscape': 2, 'Reservoir': 3, 'Trees': 3, 'Lake': 2}


{'Building': 1, 'Person': 1}


{'Building': 3, 'Grass': 1, 'Windows': 1, 'Motor': 1, 'vehicle': 1, 'City': 1, 'Trees': 2, 'Road': 3, 'Footpath': 3}


In [21]:
keys = list(result_dict.keys())

In [22]:
'Building' in result_dict[keys[0]]['result'].keys()

True

In [23]:
list_of_jsons = []
for key in keys:
    row_json = {}
    row_json['Name'] = key.split("\\")[-1]
    for i,cat in enumerate(categories):
        row_json[cat] = 1 if cat in result_dict[key]['result'].keys() else 0
    list_of_jsons.append(row_json)


In [24]:
new_keys = list(list_of_jsons[0].keys())
json_df = {}
for key in new_keys:
    json_df[key] = [x[key] for x in list_of_jsons]
    

In [25]:
import pandas as pd

In [26]:
df=pd.DataFrame(json_df)

In [27]:
df.to_csv("sub8_.csv",index=False)

In [ ]:
# res101.export("101")
# res50.export("50")
# res34.export("34")

In [ ]:
# res101 = load_learner("101")
# res50 = load_learner("50")
# res34 = load_learner("34")

In [ ]:
# res101.dls = dls
# res50.dls = dls
# res34.dls = dls
